# Test MongoDB
El objetivo de este Jupyter notebook es testear que la base de datos MongoDB esté correctamente instalada, creada completa con la última actualización de los datos que has corrido.

Para ello, ejecutaremos los siguientes tests:
1. Conexión a la base de datos MongoDB en local.
2. Revisar el tamaño y la última actualización de los datos.
3. Crear una muestra pequeña de datos para confirmar que los comandos funcionan.

Nota: La base de datos original tiene más de 24GB, por lo que cada request a la base de datos puede demorar mucho tiempo si no se ejecuta en un sistema en paralelo.

In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd


### 1. Conexión a la base de datos MongoDB en local.

In [3]:
# Reemplaza 'localhost' con la dirección de tu servidor MongoDB si es diferente
# y '27017' con el puerto correspondiente si has configurado uno distinto.
client = MongoClient('mongodb://localhost:27017/')

# Reemplaza 'nombre_de_tu_base_de_datos' con el nombre de tu base de datos
db = client['Contrataciones']

# Intenta obtener una colección para verificar la conexión
try:
    # Reemplaza 'nombre_de_tu_colección' con el nombre de una colección existente en tu base de datos
    colecciones = db.list_collection_names()  # Lista todas las colecciones para verificar la conexión
    if 'Contratos_EDCA' in colecciones:
        print("Conexión exitosa a la base de datos y la colección encontrada.")
    else:
        print("Conexión exitosa a la base de datos, pero la colección no fue encontrada.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")


# Listar todas las bases de datos
databases = client.list_database_names()
collections = db.list_collection_names()

# Imprimir la lista de bases de datos
print("Bases de datos disponibles son:", databases)

print("Colecciones dentro de la base de datos:", collections)

Conexión exitosa a la base de datos y la colección encontrada.
Bases de datos disponibles son: ['Contrataciones', 'Test', 'admin', 'config', 'local']
Colecciones dentro de la base de datos: ['Contratos_EDCA', 'Sample_Contratos_EDCA']


### 2. Revisar el tamaño y la última actualización de los datos.

Para la colección Contratos_EDCA

In [3]:
# Imprimir el tamaño de la base de datos, número de colecciones y última actualización

# Obteniendo el tamaño de la base de datos en GB
print(f"Obteniendo el tamaño de la base de datos.")
db_stats = db.command("dbstats")
db_size_gb = db_stats['storageSize'] / (1024**3)  # Convertir bytes a GB
print(f"Tamaño de la base de datos: {db_size_gb} GB")

# Obteniendo el número de documentos en la colección 'Contratos_EDCA'
print(f"Obteniendo el número de documentos. Este proceso puede tardar hasta 3 minutos...")
num_documents = db['Contratos_EDCA'].count_documents({})
print(f"Número de documentos en la colección 'Contratos_EDCA': {num_documents}")

# Obteniendo la última actualización de la base de datos
# Esto asume que hay una colección que registra las actualizaciones
# Reemplaza 'nombre_de_tu_colección_de_registro' con el nombre real de tu colección de registro

#  try: [HAY QUE CAMBIAR ESTO POR LA FECHA DEL ÚLTIMO CONTRATO]
#     last_update_collection = db['Contratos_EDCA']
#     last_update = last_update_collection.find().sort("fecha", -1).limit(1)
#     for update in last_update:
#         print(f"Última actualización: {update['fecha']}")
# except Exception as e:
#     print(f"No se pudo obtener la última actualización: {e}")


Obteniendo el tamaño de la base de datos.
Tamaño de la base de datos: 7.33831787109375 GB
Obteniendo el número de documentos. Este proceso puede tardar hasta 3 minutos...
Número de documentos en la colección 'Contratos_EDCA': 2742266


### 3. Crear una muestra pequeña de datos para confirmar que los comandos funcionan.

Guardaremos la nueva muestra _Sample_Contratos_EDCA_ en una nueva colección.

In [ ]:
# Generando la muestra aleatoria y guardándola en una nueva colección
sample_size = int(num_documents * 0.05)  # Calcula el 5% del total de documentos
print(f"Número de documentos en la muestra aleatoria es: {sample_size}")

# Realiza la agregación para obtener la muestra aleatoria
sample = db['Contratos_EDCA'].aggregate([
    { '$sample': { 'size': sample_size } }
])
sample_documents = list(sample)
print(f"Sample creado")

# Guarda la muestra en una nueva colección
db['Sample_Contratos_EDCA'].insert_many(sample_documents)
print(f"Muestra guardada en la colección 'Sample_Contratos_EDCA'!")


In [5]:
# Consulta actualizada a la base de datos Sample_Contratos_EDCA para traer sólo los 10 primeros registros con los campos license y ObjectId
consulta_actualizada = db['Sample_Contratos_EDCA'].find({}, {'license': 1, '_id': 1})
# Creando un dataframe con los resultados de la consulta actualizada


# Creando una lista de diccionarios para facilitar la creación del dataframe
datos = [{'ObjectId': contrato['_id'], 'License': contrato.get('license', 'No especificado')} for contrato in consulta_actualizada.limit(20)]

# Obtén una tabla con los 20 primeros registros de parties. Las columnas deben ser los campos de name y id
parties_data = db['Sample_Contratos_EDCA'].find({}, {'name': 1, 'id': 1}).limit(20)

# Creando el dataframe
df_contratos = pd.DataFrame(datos)

# Exportando el dataframe a un archivo csv
#df_contratos.to_csv('contratos_sample.csv', index=False)
print(df_contratos)



                    ObjectId                          License
0   639b994c0926ea7b12ed2968  https://datos.gob.mx/libreusomx
1   650a0c852fe42aff3d692726  https://datos.gob.mx/libreusomx
2   6365e96fea98b53ae64b6578  https://datos.gob.mx/libreusomx
3   6365ede4ba7e8d6512d2ab40  https://datos.gob.mx/libreusomx
4   6318f86df7e2543d58fd9788  https://datos.gob.mx/libreusomx
5   6365e7c3a149e923a0a2c28a  https://datos.gob.mx/libreusomx
6   6162e12bd3080ff91314c9ac  https://datos.gob.mx/libreusomx
7   63bd6d7e145c4f2b3955e08b  https://datos.gob.mx/libreusomx
8   634b2cb4aa1ab7afac3d5fe9  https://datos.gob.mx/libreusomx
9   6162a395c19cef8fb446ffc4  https://datos.gob.mx/libreusomx
10  62fd8856077ce42fd04d3579  https://datos.gob.mx/libreusomx
11  627dde987d9b27da8199194b  https://datos.gob.mx/libreusomx
12  6318ea4b4007f8e278a6e922  https://datos.gob.mx/libreusomx
13  6162c1c072fd56d97c75b6b2  https://datos.gob.mx/libreusomx
14  627ddfdbe646a34444e2abe1  https://datos.gob.mx/libreusomx
15  62fd

In [7]:
# Consulta actualizada a la base de datos Sample_Contratos_EDCA para traer sólo los 10 primeros registros con los campos license y ObjectId
consulta_actualizada = db['Sample_Contratos_EDCA'].find({}, {})
# Creando un dataframe con los resultados de la consulta actualizada

# Creando una lista de diccionarios para facilitar la creación del dataframe
datos = []
for contrato in consulta_actualizada.limit(20):
    contrato_dict = {
        'uri': contrato['uri'],
        'ObjectId': contrato['_id'],
        'License': contrato.get('license', 'No especificado'),
        'Publisher': contrato['publisher']['name']
    }
    datos.append(contrato_dict)

# Creando el dataframe
df_contratos = pd.DataFrame(datos)

# Exportando el dataframe a un archivo csv
#df_contratos.to_csv('contratos_sample.csv', index=False)
print(df_contratos)

                                                  uri  \
0   http://172.23.106.220:8080/OCDSWS/rest/ocid/-2...   
1   http://172.23.106.220:8080/OCDSWS/rest/ocid/-2...   
2   http://172.23.106.220:8080/OCDSWS/rest/ocid/-2...   
3   http://172.23.106.220:8080/OCDSWS/rest/ocid/-2...   
4   http://172.23.106.220:8080/OCDSWS/rest/ocid/oc...   
5   http://172.23.106.220:8080/OCDSWS/rest/ocid/-2...   
6   http://172.23.106.220:8080/OCDSWS/rest/ocid/oc...   
7   http://172.23.106.220:8080/OCDSWS/rest/ocid/-2...   
8   http://172.23.106.220:8080/OCDSWS/rest/ocid/-2...   
9   http://172.23.106.220:8080/OCDSWS/rest/ocid/oc...   
10  http://172.23.106.220:8080/OCDSWS/rest/ocid/oc...   
11  http://172.23.106.220:8080/OCDSWS/rest/ocid/oc...   
12  http://172.23.106.220:8080/OCDSWS/rest/ocid/oc...   
13  http://172.23.106.220:8080/OCDSWS/rest/ocid/oc...   
14  http://172.23.106.220:8080/OCDSWS/rest/ocid/oc...   
15  http://172.23.106.220:8080/OCDSWS/rest/ocid/oc...   
16  http://172.23.106.220:8080/